# Eat Safe, Love

## Notebook Set Up

In [1]:
from pymongo import MongoClient
import pandas as pd
from pprint import pprint

In [2]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [3]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [4]:
# review the collections in our database
print(db.list_collection_names())

['establishments']


In [5]:
# assign the collection to a variable
establishments = db['establishments']

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [6]:
# query documents with a scores Hygiene rating equal to 20
q1_query = {"scores.Hygiene": 20}
q1_fields = {"_id": 0, "BusinessName": 1, "scores.Hygiene": 1}
q1_results = list(establishments.find(q1_query, q1_fields))

In [7]:
# counn_documents() method to count the number of documents that match the query
print(establishments.count_documents(q1_query))

41


In [8]:
# print the first result
pprint(list(q1_results)[0])

{'BusinessName': 'The Chase Rest Home', 'scores': {'Hygiene': 20}}


In [9]:
# create a dataframe from the results
q1_df = pd.DataFrame(list(q1_results))

In [10]:
# print the shape of the dataframe
print(q1_df.shape)

(41, 2)


In [11]:
# display the first 10 rows
print(q1_df.head(10))

                BusinessName           scores
0        The Chase Rest Home  {'Hygiene': 20}
1                 Brenalwood  {'Hygiene': 20}
2              Melrose Hotel  {'Hygiene': 20}
3              Seaford Pizza  {'Hygiene': 20}
4              Golden Palace  {'Hygiene': 20}
5           Ashby's Butchers  {'Hygiene': 20}
6  South Sea Express Cuisine  {'Hygiene': 20}
7              Golden Palace  {'Hygiene': 20}
8             The Tulip Tree  {'Hygiene': 20}
9                      F & S  {'Hygiene': 20}


### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [12]:
# Find the establishments with London as the Local Authority and has a RatingValue greater than or equal to 4 using Regex to find the establishments with London as the Local Authority
q2_query = {'LocalAuthorityName': {'$regex': 'London', '$options': 'i'}, 'RatingValue': {'$gte': 4}}
q2_fields = {'_id': 0, 'BusinessName': 1, 'RatingValue': 1, 'LocalAuthorityName': 1}
q2_result = list(establishments.find(q2_query, q2_fields))

In [13]:
# Use count_documents to display the number of documents in the result
print(establishments.count_documents(q2_query))

33


In [14]:
# print the first result
pprint(q2_result[0])

{'BusinessName': "Charlie's",
 'LocalAuthorityName': 'City of London Corporation',
 'RatingValue': 4}


In [15]:
# Convert the result to a Pandas DataFrame
q2_df = pd.DataFrame(list(q2_result))

In [16]:
# Display the number of rows in the DataFrame
print(q2_df.shape[0])

33


In [17]:
# Display the first 10 rows of the DataFrame
q2_df.head(10)

,BusinessName,RatingValue,LocalAuthorityName
0,Charlie's,4,City of London Corporation
1,Mv City Cruises Erasmus,5,City of London Corporation
2,Benfleet Motor Yacht Club,4,City of London Corporation
3,Coombs Catering t/a The Lock and Key,5,City of London Corporation
4,Tilbury Seafarers Centre,5,City of London Corporation
5,Mv Valulla,5,City of London Corporation
6,Tereza Joanne,5,City of London Corporation
7,Brick Lane Brews,4,City of London Corporation
8,The Nuance Group (UK) Limited,5,City of London Corporation
9,WH Smith,5,City of London Corporation


### 3. What are the top 5 establishments with a `RatingValue` rating value of 5, sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [37]:
# Penang Flavours latitude and longitude
degree_search = 0.01
latitude = 51.490142
longitude = 0.08384

In [41]:
# Question 3 query
q3_query = {"RatingValue": 5, "geocode.latitude": {"$lte": latitude + degree_search, "$gte": latitude - degree_search}, "geocode.longitude": {"$lte": longitude + degree_search, "$gte": longitude - degree_search}}

In [42]:
# fields to return
q3_fields = {"_id": 0, "BusinessName": 1, "RatingValue": 1, "geocode.latitude": 1, "geocode.longitude": 1, "scores.Hygiene": 1}

In [43]:
# execute the query and sort the results by RatingValue and limit the results to 5
query3_results = list(establishments.find(q3_query, q3_fields).sort("scores.Hygiene", -1).limit(5))

In [44]:
# print all the results
pprint(query3_results)

[{'BusinessName': 'Lucky Food & Wine',
  'RatingValue': 5,
  'geocode': {'latitude': 51.4878934, 'longitude': 0.0910104},
  'scores': {'Hygiene': 5}},
 {'BusinessName': 'Everest Stores Ltd',
  'RatingValue': 5,
  'geocode': {'latitude': 51.4876718, 'longitude': 0.0903232},
  'scores': {'Hygiene': 5}},
 {'BusinessName': 'Premier Express',
  'RatingValue': 5,
  'geocode': {'latitude': 51.4877234, 'longitude': 0.0904552},
  'scores': {'Hygiene': 5}},
 {'BusinessName': 'Fineway Cash & Carry',
  'RatingValue': 5,
  'geocode': {'latitude': 51.4876089, 'longitude': 0.0909382},
  'scores': {'Hygiene': 5}},
 {'BusinessName': 'TIWA N TIWA African Restaurant Ltd',
  'RatingValue': 5,
  'geocode': {'latitude': 51.4870351, 'longitude': 0.0927429},
  'scores': {'Hygiene': 5}}]


In [45]:
# convert the q3_results to a DataFrame
q3_df = pd.DataFrame(list(query3_results))

q3_df

,BusinessName,RatingValue,scores,geocode
0,Lucky Food & Wine,5,{'Hygiene': 5},"{'longitude': 0.0910104, 'latitude': 51.4878934}"
1,Everest Stores Ltd,5,{'Hygiene': 5},"{'longitude': 0.0903232, 'latitude': 51.4876718}"
2,Premier Express,5,{'Hygiene': 5},"{'longitude': 0.0904552, 'latitude': 51.4877234}"
3,Fineway Cash & Carry,5,{'Hygiene': 5},"{'longitude': 0.0909382, 'latitude': 51.4876089}"
4,TIWA N TIWA African Restaurant Ltd,5,{'Hygiene': 5},"{'longitude': 0.0927429, 'latitude': 51.4870351}"


### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [46]:
# Create a pipeline that:
# 1. Matches establishments with a hygiene score of 0
q4_query = {"scores.Hygiene": 0}

# Groups the matches by Local Authority
q4_pipeline = [{"$match": q4_query}, {"$group": {"_id": "$LocalAuthorityName", "count": {"$sum": 1}}}]

# Sorts the matches from highest to lowest
q4_pipeline.append({"$sort": {"count": -1}})

# the number of documents in the result
print(establishments.count_documents(q4_query))

# the first ten results
print(list(establishments.aggregate(q4_pipeline))[:10])

16827
[{'_id': 'Thanet', 'count': 1130}, {'_id': 'Greenwich', 'count': 882}, {'_id': 'Maidstone', 'count': 713}, {'_id': 'Newham', 'count': 711}, {'_id': 'Swale', 'count': 686}, {'_id': 'Chelmsford', 'count': 680}, {'_id': 'Medway', 'count': 672}, {'_id': 'Bexley', 'count': 607}, {'_id': 'Southend-On-Sea', 'count': 586}, {'_id': 'Tendring', 'count': 542}]


In [47]:
# data frame creation
q4_df = pd.DataFrame(list(establishments.aggregate(q4_pipeline)))

# the number of rows in the DataFrame
print(q4_df.shape[0])

# first 10 rows in the data frame
q4_df.head(10)

55


,_id,count
0,Thanet,1130
1,Greenwich,882
2,Maidstone,713
3,Newham,711
4,Swale,686
5,Chelmsford,680
6,Medway,672
7,Bexley,607
8,Southend-On-Sea,586
9,Tendring,542
